# Homework 4

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2025

***

### Data

The file `data/ratetree_data_2025-01-31.xlsx` has a binomial tree of interest rates fit to...
* discount curves from `cap_curves_2025-01-31.xlsx`
* implied vols from `cap_curves_2025-01-31.xlsx`

Note the following...
* Suppose the present date is `2025-01-31`.
* The rates are continuously compounded.
* The rates are for the following quarter. So teh rate at $t=0$ is the continuously compounded rate for the interval $t=0$ to $t=.25$.

Take this binomial tree as given; there is no need to fit it yourself.

In [3]:
import pandas as pd
import numpy as np

DATE = '2025-01-31'
FILEIN = f'../data/ratetree_data_{DATE}.xlsx'
sheet_tree = 'rate tree'

ratetree = pd.read_excel(FILEIN, sheet_name=sheet_tree).set_index('state')
ratetree.columns.name = 'time'

ratetree.style.format('{:.1%}',na_rep='').format_index('{:.2f}',axis=1)

time,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75
state,,,,,,,,,,,,,,,,,,,,
0,4.2%,4.3%,4.4%,4.8%,5.1%,6.4%,7.6%,9.1%,10.1%,11.8%,13.5%,15.3%,16.9%,19.2%,22.7%,25.9%,28.2%,30.6%,34.7%,40.3%
1,,3.9%,4.0%,4.2%,4.4%,5.2%,6.0%,7.0%,7.8%,9.0%,10.4%,11.8%,13.0%,14.8%,17.4%,19.9%,21.7%,23.7%,27.0%,31.2%
2,,,3.6%,3.7%,3.8%,4.2%,4.7%,5.4%,6.0%,7.0%,8.0%,9.1%,10.1%,11.4%,13.4%,15.3%,16.7%,18.4%,20.9%,24.2%
3,,,,3.3%,3.2%,3.3%,3.7%,4.2%,4.6%,5.3%,6.2%,7.0%,7.8%,8.8%,10.3%,11.8%,12.9%,14.2%,16.3%,18.8%
4,,,,,2.8%,2.7%,2.9%,3.3%,3.6%,4.1%,4.7%,5.4%,6.0%,6.8%,7.9%,9.1%,10.0%,11.0%,12.6%,14.6%
5,,,,,,2.2%,2.3%,2.5%,2.7%,3.2%,3.6%,4.2%,4.6%,5.2%,6.1%,7.0%,7.7%,8.6%,9.8%,11.3%
6,,,,,,,1.8%,2.0%,2.1%,2.4%,2.8%,3.2%,3.6%,4.0%,4.7%,5.4%,5.9%,6.6%,7.6%,8.8%
7,,,,,,,,1.5%,1.6%,1.9%,2.2%,2.5%,2.7%,3.1%,3.6%,4.1%,4.6%,5.1%,5.9%,6.8%
8,,,,,,,,,1.3%,1.4%,1.7%,1.9%,2.1%,2.4%,2.8%,3.2%,3.5%,4.0%,4.6%,5.3%


***

# 1. Binomial Tree Pricing - Bond

### The Bond

Consider a vanilla (non-callable) bond with the following parameters...
* `T=5`
* coupon rate is `4.41%`
* coupons are semiannual

Note that this is essentially the hypothetical bond priced in HW 1.

### 1.1

Create and display a tree of cashflows from the bond, corresponding to each node of the tree (state and time) seen in the interest rate tree.

Note that the cashflows do not depend on the interest rates. Thus, report the cashflows at the time (in the column) they are actually paid out. The final payoff (face plus coupon) occurs at $T$, which is beyond the interest rate tree. You are welcome to add a column for $T$ or to consider this payoff separately and leave it out of the tree.

In [4]:
cf_df = pd.DataFrame({0: [0]})

for i in range(1, 21):
    if i == 20:
        payout = 100 + 2.205
    elif i % 2 == 0:
        payout = 2.205
    else:
        payout = 0
    cf_df[i / 4] = [payout]

cf_df

,0.00,0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,...,2.75,3.00,3.25,3.50,3.75,4.00,4.25,4.50,4.75,5.00
0,0,0,2.205,0,2.205,0,2.205,0,2.205,0,...,0,2.205,0,2.205,0,2.205,0,2.205,0,102.205


### 1.2.

Create and display a tree of values of the bond. Do this for the quotes as
* clean quotes
* dirty quotes

Given the semiannual coupons and quarterly tree steps, the clean and dirty will coincide at $t=0, .5, 1,...$.

Do the valuation by...
* setting the value at $T$ as the face plus final coupon.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

In [20]:
def price_vanilla_bond(ratetree, cpn_rate, face=100, freq=2, clean=True):
    cpn = cpn_rate * face / freq
    disc_tree = np.power(1 - ratetree, -0.25) # discount tree
    val_tree = ratetree.copy()
    

    for i in range(19, -1, -1):
        time = i / 4
        if i == 19:
            val_tree[time] = (face + cpn) / disc_tree[time]
            continue
        
        avg_series = val_tree[time + 0.25].rolling(window=2).mean()
        avg_series = avg_series.shift(-1)
        
        if i % 2 == 0:
            val_tree[time] = cpn + avg_series / disc_tree[time]
        else:
            val_tree[time] = avg_series / disc_tree[time]

        
    if not clean: # add accrued interest
        for t in range(1, 21, 2):
            ti = t / 4
            val_tree[ti] += cpn / 2
    
    return val_tree
    


In [21]:
# clean price tree
price_vanilla_bond(ratetree, 0.0441)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75
state,,,,,,,,,,,,,,,,,,,,
0,103.649623,100.532243,99.470072,96.006539,94.621848,90.863382,89.451697,85.907727,84.764308,81.479004,80.663642,77.873158,77.655484,75.461442,76.050156,75.281074,77.883040,79.263026,84.409490,89.841923
1,NaN,104.551354,103.828569,100.748832,99.758409,96.409247,95.324641,92.077030,91.176696,88.116508,87.477800,84.835864,84.713496,82.593948,83.156436,82.204942,84.382240,85.136004,89.299225,93.068931
2,NaN,NaN,107.367680,104.593001,103.915680,100.892870,100.071204,97.063188,96.360424,93.482534,92.983119,90.449715,90.383611,88.291691,88.789239,87.633544,89.406608,89.589537,92.907610,95.357993
3,NaN,NaN,NaN,107.685879,107.250860,104.481605,103.866643,101.048408,100.503306,97.771027,97.380729,94.926953,94.893056,92.803730,93.224335,91.874588,93.293226,92.988547,95.609645,97.025665
4,NaN,NaN,NaN,NaN,109.909004,107.332426,106.877215,104.207099,103.786237,101.169237,100.864190,98.469311,98.453242,96.354399,96.699735,95.179380,96.300832,95.593969,97.652717,98.261907
5,NaN,NaN,NaN,NaN,NaN,109.584121,109.250715,106.694934,106.371112,103.844836,103.606347,101.255448,101.248843,99.135673,99.413628,97.749799,98.628769,97.597128,99.207966,99.189277
6,NaN,NaN,NaN,NaN,NaN,NaN,111.113306,108.645037,108.396526,105.941360,105.754844,103.437106,103.435236,101.306827,101.527419,99.746276,100.430914,99.140590,100.397580,99.890813
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.168077,109.977741,107.578153,107.432242,105.139730,105.140008,102.997360,103.170641,101.295369,101.826177,100.331742,101.310719,100.424749
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.208722,108.852483,108.738314,106.465140,106.466211,104.311123,104.446206,102.496403,102.906509,101.252088,102.013465,100.832950


In [22]:
# dirty price tree
price_vanilla_bond(ratetree, 0.0441, clean=False)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3,3.25,3.5,3.75,4,4.25,4.5,4.75
state,,,,,,,,,,,,,,,,,,,,
0,103.649623,101.634743,99.470072,97.109039,94.621848,91.965882,89.451697,87.010227,84.764308,82.581504,80.663642,78.975658,77.655484,76.563942,76.050156,76.383574,77.883040,80.365526,84.409490,90.944423
1,NaN,105.653854,103.828569,101.851332,99.758409,97.511747,95.324641,93.179530,91.176696,89.219008,87.477800,85.938364,84.713496,83.696448,83.156436,83.307442,84.382240,86.238504,89.299225,94.171431
2,NaN,NaN,107.367680,105.695501,103.915680,101.995370,100.071204,98.165688,96.360424,94.585034,92.983119,91.552215,90.383611,89.394191,88.789239,88.736044,89.406608,90.692037,92.907610,96.460493
3,NaN,NaN,NaN,108.788379,107.250860,105.584105,103.866643,102.150908,100.503306,98.873527,97.380729,96.029453,94.893056,93.906230,93.224335,92.977088,93.293226,94.091047,95.609645,98.128165
4,NaN,NaN,NaN,NaN,109.909004,108.434926,106.877215,105.309599,103.786237,102.271737,100.864190,99.571811,98.453242,97.456899,96.699735,96.281880,96.300832,96.696469,97.652717,99.364407
5,NaN,NaN,NaN,NaN,NaN,110.686621,109.250715,107.797434,106.371112,104.947336,103.606347,102.357948,101.248843,100.238173,99.413628,98.852299,98.628769,98.699628,99.207966,100.291777
6,NaN,NaN,NaN,NaN,NaN,NaN,111.113306,109.747537,108.396526,107.043860,105.754844,104.539606,103.435236,102.409327,101.527419,100.848776,100.430914,100.243090,100.397580,100.993313
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.270577,109.977741,108.680653,107.432242,106.242230,105.140008,104.099860,103.170641,102.397869,101.826177,101.434242,101.310719,101.527249
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.208722,109.954983,108.738314,107.567640,106.466211,105.413623,104.446206,103.598903,102.906509,102.354588,102.013465,101.935450


### 1.3.

The binomial-estimated price of the bond is the initial node of the value tree.

Report this along with the price of the bond you would get from the usual simple formula for such a bond. 
* Consider pricing it with the $T$ interval swap rate (used similar to a ytm) from the file `cap_curves_2025-01-31.xlsx`.
* If you do this, recall that the swap rate given in that file is quarterly-compounded, so you would need to convert it to semiannual compounding before plugging it into the usual closed-form ytm-pricing formula.

In [23]:
cap_path = "../data/cap_curves_2025-01-31.xlsx"
swap_df = pd.read_excel(cap_path).set_index('tenor')

In [24]:
swap_df['discount rates'] = np.exp(- swap_df['swap rates'] / 4).cumprod()

cpn = 4.41 / 2
swap_price = cpn
for t in range(1, 11):
    swap_price += cpn * swap_df['discount rates'].loc[t / 2]
    if t == 10:
        swap_price += 100 * swap_df['discount rates'].loc[t / 2]

print(f"The price based upon swaps is {round(swap_price, 2)}")

The price based upon swaps is 103.78


### Note:

An easy check on your code is whether it will correctly price a zero-coupon bond at a price that matches the "discounts" in the `cap_curves` data file.

***

# 2. Pricing the Callable - European

### 2.1.

Calculate and display value tree of a European-style call option on the bond analyzed in part `1`.
* `$T_o = 3$`. That is, the time-to-expiration is 3 years.
* `$K=100$`. That is, the strike is 100. This is a clean strike, meaning exercise requires paying the strike plus any accrued interest.

Do so by 
* setting the value at the time of expiration, using the value of the bond for each node at that time.
* discounting this back through time, using the (continuously-compounded) interest rate.
* recall that the tree is constructed such that the probability of moving "up" or "down" is 50%.

Note that...
* the tree of call values will not be the same size as the tree of bond values. The former goes only to $T_o=3$.

In [25]:
def price_bond_call(ratetree, cpn, expiry, strike, freq=2, eur=True):
    disc_tree = np.exp(ratetree / 4)
    clean_tree = price_vanilla_bond(ratetree, cpn, freq=2)
    option_tree = clean_tree.copy()
    option_tree = option_tree[np.arange(0, 3 + 0.25, 0.25)]

    if eur:
        option_tree[expiry] = np.maximum(0, option_tree[expiry] - strike)

        for t in np.arange(expiry - 0.25, -0.25, -0.25):
            avg_series = option_tree[t + 0.25].rolling(window=2).mean()
            avg_series = avg_series.shift(-1)
            option_tree[t] = avg_series / disc_tree[t]

    else:
        option_tree = np.maximum(0, option_tree - strike)

        for t in np.arange(expiry - 0.25, -0.25, -0.25):
            avg_series = option_tree[t + 0.25].rolling(window=2).mean()
            avg_series = avg_series.shift(-1)

            option_tree[t] = np.maximum(avg_series / disc_tree[t], option_tree[t])

    return option_tree
        

In [26]:
price_bond_call(ratetree, 0.0441, 3, 100)

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,3.027747,2.400567,1.792184,1.233248,0.757438,0.393022,0.155492,0.035722,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,3.719023,3.061131,2.391198,1.738728,1.141325,0.643309,0.281242,0.073079,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,4.449912,3.792879,3.094224,2.374391,1.669050,1.024805,0.499368,0.149041,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,5.188092,4.562011,3.872348,3.129313,2.352900,1.578344,0.864825,0.303308,0.000000,0.000000
4,NaN,NaN,NaN,NaN,5.898905,5.325259,4.680266,3.964086,3.177422,2.328612,1.449601,0.616016,0.000000
5,NaN,NaN,NaN,NaN,NaN,6.554027,6.041861,5.465066,4.815964,4.083107,3.255676,2.317694,1.248843
6,NaN,NaN,NaN,NaN,NaN,NaN,7.136936,6.688312,6.183835,5.615115,4.975213,4.253219,3.435236
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.650269,7.258871,6.818031,6.323306,5.767177,5.140008
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.100260,7.758841,7.376435,6.947815,6.466211


### 2.2.

Show the value tree of the callable bond by subtracting the call value tree from the (subset $t\le T_o$ of the) bond value tree (calculated in part `1`.) Do this for both
* clean
* dirty

In [27]:
call_tree = price_bond_call(ratetree, 0.0441, 3, 100)
clean_tree = price_vanilla_bond(ratetree, 0.0441)
dirty_tree = price_vanilla_bond(ratetree, 0.0441, clean=False)

clean_callable = clean_tree[np.arange(0, 3.25, 0.25)] - call_tree
dirty_callable = clean_tree[np.arange(0, 3.25, 0.25)] - call_tree

In [28]:
# clean callable price tree
clean_callable

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,100.621877,98.131675,97.677888,94.773291,93.864411,90.470360,89.296205,85.872005,84.764308,81.479004,80.663642,77.873158,77.655484
1,NaN,100.832331,100.767438,98.357634,98.019682,95.267922,94.681331,91.795788,91.103617,88.116508,87.477800,84.835864,84.713496
2,NaN,NaN,102.917767,100.800122,100.821456,98.518479,98.402154,96.038382,95.861056,93.333494,92.983119,90.449715,90.383611
3,NaN,NaN,NaN,102.497787,102.688849,100.609257,100.737330,98.695508,98.924961,96.906202,97.077421,94.926953,94.893056
4,NaN,NaN,NaN,NaN,104.010099,102.007167,102.196949,100.243013,100.608815,98.840625,99.414590,97.853295,98.453242
5,NaN,NaN,NaN,NaN,NaN,103.030094,103.208853,101.229868,101.555147,99.761729,100.350671,98.937754,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,103.976371,101.956725,102.212691,100.326245,100.779631,99.183888,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.517809,102.718870,100.760121,101.108936,99.372554,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.108462,101.093642,101.361879,99.517326,100.000000


In [29]:
# dirty callable price tree
dirty_callable

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,100.621877,98.131675,97.677888,94.773291,93.864411,90.470360,89.296205,85.872005,84.764308,81.479004,80.663642,77.873158,77.655484
1,NaN,100.832331,100.767438,98.357634,98.019682,95.267922,94.681331,91.795788,91.103617,88.116508,87.477800,84.835864,84.713496
2,NaN,NaN,102.917767,100.800122,100.821456,98.518479,98.402154,96.038382,95.861056,93.333494,92.983119,90.449715,90.383611
3,NaN,NaN,NaN,102.497787,102.688849,100.609257,100.737330,98.695508,98.924961,96.906202,97.077421,94.926953,94.893056
4,NaN,NaN,NaN,NaN,104.010099,102.007167,102.196949,100.243013,100.608815,98.840625,99.414590,97.853295,98.453242
5,NaN,NaN,NaN,NaN,NaN,103.030094,103.208853,101.229868,101.555147,99.761729,100.350671,98.937754,100.000000
6,NaN,NaN,NaN,NaN,NaN,NaN,103.976371,101.956725,102.212691,100.326245,100.779631,99.183888,100.000000
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.517809,102.718870,100.760121,101.108936,99.372554,100.000000
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,103.108462,101.093642,101.361879,99.517326,100.000000


### 2.3.

Report the initial node value of the call option and of the callable bond.

In a table, compare these to what you got in HW 1 as the value of the embedded call and the value of the callable bond.
* In `HW 1`, we were valuing from a date nearly two weeks later, `2025-02-13`. This difference in the timing means we wouldn't expect the values to match exactly, even if the methods were entirely consistent.

***

In [30]:
callable_init = clean_callable[0].loc[0]
call_init = call_tree[0].loc[0]

print(f"Callable bond value: {callable_init}")
print(f"Call value: {call_init}")


Callable bond value: 100.62187679252501
Call value: 3.0277465871537097


**Answer**

The call value that we calculated is 2.90. The callable was 100.9 for market value. The calculated value is 98.67. Both of these are similar.

# 3. Pricing the Callable - American

### 3.1.

Re-do part `2.`, but this time, make the option a **American** style. That is, allow it to be exercised at any node.
* Report the tree of callable-bond values.
* How does this compare to the European-style?

#### Note
To do this valuation, go through the procedure in `2.1.`, but at each node, compare the value for the call with the value of the payoff function based on the vanilla bond's value at that node. Take the maximum of the two. If you code this carefully, you can simply add a line of code to what you did in `2.1`.

In [31]:
amer_call_tree = price_bond_call(ratetree, 0.0441, 3, 100, eur=False)

In [32]:
amer_call_tree

time,0,0.25,0.5,0.75,1,1.25,1.5,1.75,2,2.25,2.5,2.75,3
state,,,,,,,,,,,,,
0,4.419938,3.179330,2.174275,1.398936,0.814045,0.403275,0.155492,0.035722,0.000000,0.000000,0.000000,0.000000,0.000000
1,NaN,5.754115,4.253494,2.998237,2.017483,1.245742,0.664148,0.281242,0.073079,0.000000,0.000000,0.000000,0.000000
2,NaN,NaN,7.367680,5.594642,4.042381,2.833618,1.859764,1.067112,0.499368,0.149041,0.000000,0.000000,0.000000
3,NaN,NaN,NaN,8.510435,7.250860,5.327298,3.866643,2.696546,1.664118,0.864825,0.303308,0.000000,0.000000
4,NaN,NaN,NaN,NaN,9.909004,8.010108,6.877215,5.037241,3.786237,2.502156,1.449601,0.616016,0.000000
5,NaN,NaN,NaN,NaN,NaN,10.127355,9.250715,7.337053,6.371112,4.643817,3.606347,2.317694,1.248843
6,NaN,NaN,NaN,NaN,NaN,NaN,11.113306,9.141971,8.396526,6.553691,5.754844,4.253219,3.435236
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.552820,9.977741,8.047696,7.432242,5.767177,5.140008
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.208722,9.212700,8.738314,6.947815,6.466211


### 3.2.

In which nodes will the American-style callable bond be exercised?

In [33]:
price_diff = clean_tree - amer_call_tree

proc_df = np.where(pd.isna(price_diff), np.nan, price_diff == 100)
proc_df = pd.DataFrame(proc_df, index=clean_tree.index, columns=clean_tree.columns)

def highlight_cells(val):
    if pd.isna(val):
        return ''
    elif val:
        return 'background-color: green'
    else:
        return 'background-color: red'

styled_proc_df = proc_df.style.applymap(highlight_cells)
styled_proc_df

C:\Users\xuv14\AppData\Local\Temp\ipykernel_38792\789674499.py:14: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_proc_df = proc_df.style.applymap(highlight_cells)


time,0,0.250000,0.500000,0.750000,1,1.250000,1.500000,1.750000,2,2.250000,2.500000,2.750000,3,3.250000,3.500000,3.750000,4,4.250000,4.500000,4.750000
state,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
1,nan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
2,nan,nan,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
3,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
4,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan
5,nan,nan,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
6,nan,nan,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
7,nan,nan,nan,nan,nan,nan,nan,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan
8,nan,nan,nan,nan,nan,nan,nan,nan,1.000000,0.000000,1.000000,0.000000,1.000000,nan,nan,nan,nan,nan,nan,nan


***

# 4. Pricing the Callable - Bermudan

#### This Section is NOT REQUIRED and NOT EXPECTED
Still, it is not much additional work, and some of you may find it interesting. It also illustrates the power of binomial trees in how easily they handle the Bermudan style. 

### 4.1.

Re-do part `3`, but this time with **Bermudan** style exercise. 
* This corresponds to the Freddie Mac bond in `HW 1`.
* Note that the option value tree will now go all the way to $T$.

As a reminder, the Bermudan style can be exercised as early as $T_o$ all the way to $T$. It can only be exercised on specific dates at 3-month intervals, but in our quarterly-spaced tree, this means every node from $T_o$ onward.

### 4.2.

Compare the valuation to the market quote in `HW 1`.

***